# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [1]:
%pip install sentence-transformers
%pip install faiss-cpu
%pip install openai
%pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 26.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 91.0 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=ebd838cc1f676dcf45dcb90eedb6d22fcbfe298f9be6d4d94230bda9eb2e62aa
  Stored

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss
from openai import OpenAI
from google.colab import userdata
import yfinance as yf

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [3]:
# We define the directory where the data is located
from google.colab import drive
drive.mount('/content/drive')
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/Fintech//Data"
os.chdir(DIR)

Mounted at /content/drive


In [4]:
K = 25

In [5]:
df_news = pd.read_csv('df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [6]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.


In [8]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [9]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [10]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [11]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [12]:
# Initializing the OpenAI client with API key
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# Sending a test request
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Write a one-sentence bedtime story about a unicorn."}
        ],
        max_tokens=50
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")

As the stars twinkled brightly above the enchanted forest, Luna the unicorn spread her shimmering wings and soared through the night sky, painting dreams with her magical horn for all the children asleep below.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [13]:
# Getting unique tickers from df_news
tickers = df_news['TICKER'].unique().tolist()

# Creating an empty list to store metadata
metadata_list = []

# Iterating over tickers and retrieve metadata
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        metadata = {
            'TICKER': ticker,
            'COMPANY_NAME': info.get('longName', 'N/A'),
            'SECTOR': info.get('sector', 'N/A'),
            'INDUSTRY': info.get('industry', 'N/A')
        }
        metadata_list.append(metadata)
    except Exception as e:
        print(f"Could not retrieve data for {ticker}: {e}")
        metadata_list.append({
            'TICKER': ticker,
            'COMPANY_NAME': 'N/A',
            'SECTOR': 'N/A',
            'INDUSTRY': 'N/A'
        })

# Creating the df_meta DataFrame
df_meta = pd.DataFrame(metadata_list)

# Displaying the first few rows of df_meta
display(df_meta.head())

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [14]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [15]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [16]:
# CODE HERE
# Use as many coding cells as you need

def generate_rag_answer(query, vector_store, openai_client, k=5):
    """
    Generates an answer to a query using Retrieval-Augmented Generation (RAG).

    Args:
        query (str): The user's query.
        vector_store (FaissVectorStore): The FAISS vector store instance.
        openai_client: The initialized OpenAI client.
        k (int): The number of relevant documents to retrieve.

    Returns:
        tuple: A tuple containing the generated answer (str) and a list of
               retrieved documents with their metadata and similarity scores.
    """
    # 1. Retrieving relevant documents
    retrieved_docs = vector_store.search(query, k=k)

    if not retrieved_docs:
        return "Could not find relevant information.", []

    # 2. Constructing the prompt
    context = "\n\n".join([f"Document {i+1}:\n{doc[0]}" for i, doc in enumerate(retrieved_docs)])
    prompt = f"""Based on the following financial news articles, answer the question:

News Articles:
{context}

Question: {query}

Answer:
"""

    # 3. Generating answer using OpenAI model
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes financial news."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.7
        )
        answer = response.choices[0].message.content
    except Exception as e:
        answer = f"An error occurred while generating the answer: {e}"

    return answer, retrieved_docs

# Function to process the list of questions
def process_questions(question_list, title, vector_store, openai_client, k=K):
    print_markdown(f"# {title}\n")
    for query in question_list:
        answer, retrieved_documents = generate_rag_answer(query, vector_store, openai_client, k=k)

        print_markdown(f"## Question:\n{query}")
        print_markdown(f"## Generated Answer:\n{answer}")
        print_markdown("## Retrieved Documents:")
        for i, (doc, meta, sim) in enumerate(retrieved_documents):
            print_markdown(f"**Document {i+1} (Similarity: {sim:.4f}):**")
            print_markdown(f"- **Ticker:** {meta['TICKER']}")
            print_markdown(f"- **Date:** {meta['PUBLICATION_DATE']}")
            print_markdown(f"- **Provider:** {meta['PROVIDER']}")
            print_markdown(f"- **Content:** {doc[:300]}...") # Print first 300 characters of the document
        print_markdown("---\n") # Separator

# Processig all question lists
process_questions(questions_topic, "Topic-Focused Questions", faiss_store, client)
process_questions(questions_company, "Company-Focused Questions", faiss_store, client)
process_questions(questions_industry, "Industry-Focused Questions", faiss_store, client)

# Topic-Focused Questions


## Question:
What are the major concerns expressed in financial news about inflation?

## Generated Answer:
The major concerns expressed in financial news about inflation include:

1. **Persistent Inflation Risks**: The Federal Reserve's minutes from the May policy meeting highlighted growing concerns about ongoing inflation in the U.S. and the potential for an economic slowdown, indicating that inflation remains a significant issue for policymakers.

2. **Food Inflation**: There is a notable impact of food inflation on consumer expectations and monetary policy decisions. Rising food prices have dampened hopes for a rate cut, suggesting that inflationary pressures are affecting overall economic sentiment and monetary policy considerations.

3. **Macro Economic Uncertainty**: The overall macroeconomic picture continues to be uncertain, leading analysts to lower earnings expectations. This uncertainty is compounded by inflation, which is seen as a drag on economic growth and corporate profitability.

4. **Tariff Impacts**: New tariffs have contributed to rising costs, particularly in consumer goods, adding to inflationary pressures and affecting grocery bills, which have continued to increase.

5. **Long-term Fiscal Concerns**: There are growing worries about the sustainability of fiscal policies, especially with the U.S. national debt surpassing $36 trillion. This has led to increasing demand for hard assets like gold as investors seek stability amid inflation and potential currency devaluation.

Overall, these concerns reflect a complex interaction between inflation, economic growth, and fiscal policy, suggesting that inflation will continue to be a key focus for investors and policymakers in the near future.

## Retrieved Documents:

**Document 1 (Similarity: 0.5771):**

...

- **Ticker:** XOM

- **Date:** 2025-05-29

- **Provider:** MT Newswires

- **Content:** Sector Update: Energy Stocks Rise Thursday Afternoon : Energy stocks rose Thursday afternoon with the NYSE Energy Sector Index up 0.1% and the Energy Selec...

---


## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance**
2. **Completeness**  
3. **Bias or Noise**
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

YOUR WRITTEN RESPONSE HERE:
* It seems the morsels of news that the system picked up did end up substantiating what the language model produced very well. The key points about, for instance, inflation or what is happening with certain companies, came directly from the morsels of news. Generally, the news fetched was a good basis for the responses. If news was on the subject, the response came out well supported.


#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

YOUR WRITTEN RESPONSE HERE:
* I think the answers usually capture the main points of the retrieved news snippets, such as the one about inflation, which summarizes several details from the document. However, I'm not sure the answer "fully" addresses the question; I think it will depend on the complexity of the query and the completeness of the retrieved information. If the news only covers parts of a topic, the answer will be limited by that context, and the retrieved snippets might omit important details from other articles, making the answer less comprehensive. Therefore, while the answers may summarize the given context well, their completeness directly depends on how effectively the initial retrieval compiles all the necessary information.
  

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

YOUR WRITTEN RESPONSE HERE:
* When reviewing the retrieved news fragments, most seem quite relevant to the questions. However, some fragments may appear that may be off-topic, causing the language model to stray slightly or include less useful information, diluting the main points or even slightly misrepresenting things if the irrelevant information is sufficiently compelling. Therefore, keeping the retrieved fragments focused is key to a good answer.


#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

YOUR WRITTEN RESPONSE HERE:
* In news clips, slightly differing opinions or even conflicting information can sometimes be provided, especially if information is extracted from various sources or dates. The linguistic model handles this; it generally attempts to synthesize everything. It will present both opinions if they are prominent, or it will favor the opinion that is most common or most strongly presented in the clips. It does not generally express the contradiction directly, but attempts to present a single vision based on context.
  

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

YOUR WRITTEN RESPONSE HERE:
* With respect to how we could further enhance RAG's configuration, I think one of the most significant things is refining the retrieval. Perhaps including more sophisticated filtering based on metadata, not just ticker or date, but also the sentiment or credibility of the source. Tuning the k value (more or fewer documents retrieved) may also impact context quality. And generation-wise, keeping the prompt very specific in terms of the desired format of response or what type of information to highlight from the pieces of text would help achieve even more specific and correct answers.
  

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [17]:
# CODE HERE
# Use as many coding cells as you need

def generate_rag_answer_with_metadata(query, vector_store, openai_client, df_meta, k=5):
    """
    Generates an answer to a query using Retrieval-Augmented Generation (RAG)
    with financial metadata included in the context.

    Args:
        query (str): The user's query.
        vector_store (FaissVectorStore): The FAISS vector store instance.
        openai_client: The initialized OpenAI client.
        df_meta (pd.DataFrame): DataFrame containing company metadata.
        k (int): The number of relevant documents to retrieve.

    Returns:
        tuple: A tuple containing the generated answer (str) and a list of
               retrieved documents with their metadata and similarity scores.
    """
    # 1. Retrieving relevant documents
    retrieved_docs = vector_store.search(query, k=k)

    if not retrieved_docs:
        return "Could not find relevant information.", []

    # 2. Enriching retrieved documents with metadata and construct the context
    context_parts = []
    for i, (doc, meta, sim) in enumerate(retrieved_docs):
        ticker = meta['TICKER']
        # Getting company metadata from df_meta
        company_info = df_meta[df_meta['TICKER'] == ticker].iloc[0] if ticker in df_meta['TICKER'].values else {}

        context_part = f"Document {i+1} (Similarity: {sim:.4f}):\n"
        context_part += f"Ticker: {ticker}\n"
        if 'COMPANY_NAME' in company_info:
             context_part += f"Company Name: {company_info['COMPANY_NAME']}\n"
        if 'SECTOR' in company_info:
             context_part += f"Sector: {company_info['SECTOR']}\n"
        if 'INDUSTRY' in company_info:
             context_part += f"Industry: {company_info['INDUSTRY']}\n"
        context_part += f"Content: {doc}\n"
        context_parts.append(context_part)

    context = "\n\n".join(context_parts)

    # 3. Constructing the prompt
    prompt = f"""Based on the following financial news articles and their associated metadata, answer the question.
Focus on providing a concise and informative answer grounded in the provided context.

News Articles with Metadata:
{context}

Question: {query}

Answer:
"""

    # 4. Generating answer using OpenAI model
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes financial news with company context."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,
            temperature=0.7
        )
        answer = response.choices[0].message.content
    except Exception as e:
        answer = f"An error occurred while generating the answer: {e}"

    return answer, retrieved_docs

# Function to process the list of questions with metadata
def process_questions_with_metadata(question_list, title, vector_store, openai_client, df_meta, k=K):
    print_markdown(f"# {title} (With Metadata)\n")
    for query in question_list:
        answer, retrieved_documents = generate_rag_answer_with_metadata(query, vector_store, openai_client, df_meta, k=k)

        print_markdown(f"## Question:\n{query}")
        print_markdown(f"## Generated Answer (With Metadata):\n{answer}")
        print_markdown("## Retrieved Documents with Metadata:")
        for i, (doc, meta, sim) in enumerate(retrieved_documents):
            ticker = meta['TICKER']
            company_info = df_meta[df_meta['TICKER'] == ticker].iloc[0] if ticker in df_meta['TICKER'].values else {}
            print_markdown(f"**Document {i+1} (Similarity: {sim:.4f}):**")
            print_markdown(f"- **Ticker:** {ticker}")
            if 'COMPANY_NAME' in company_info:
                 print_markdown(f"- **Company Name:** {company_info['COMPANY_NAME']}")
            if 'SECTOR' in company_info:
                 print_markdown(f"- **Sector:** {company_info['SECTOR']}")
            if 'INDUSTRY' in company_info:
                 print_markdown(f"- **Industry:** {company_info['INDUSTRY']}")
            print_markdown(f"- **Date:** {meta['PUBLICATION_DATE']}")
            print_markdown(f"- **Provider:** {meta['PROVIDER']}")
            print_markdown(f"- **Content:** {doc[:300]}...") # Print first 300 characters
        print_markdown("---\n") # Separator


# Processing all question lists with metadata
process_questions_with_metadata(questions_topic, "Topic-Focused Questions", faiss_store, client, df_meta)
process_questions_with_metadata(questions_company, "Company-Focused Questions", faiss_store, client, df_meta)
process_questions_with_metadata(questions_industry, "Industry-Focused Questions", faiss_store, client, df_meta)

# Topic-Focused Questions (With Metadata)


## Question:
What are the major concerns expressed in financial news about inflation?

## Generated Answer (With Metadata):
The financial news articles highlight several major concerns about inflation, particularly its persistent nature and its implications for economic policy and consumer behavior:

1. **Persistent Inflation Risks**: The Federal Reserve's recent minutes indicate growing concern over ongoing inflation in the U.S., suggesting that it could lead to economic slowdown. This concern is central to the Fed's policy discussions and decisions.

2. **Impact on Interest Rates**: Articles suggest that food inflation dampens expectations for potential rate cuts by the Fed. Analysts believe that ongoing inflation may prevent the Fed from lowering interest rates, which could affect borrowing costs and economic growth.

3. **Consumer Price Pressures**: Rising prices in essential goods, such as groceries, are noted as a significant issue, leading consumers to seek strategies to mitigate the impact of inflation on their budgets. This has implications for consumer spending patterns and overall economic sentiment.

4. **Geopolitical and Macroeconomic Factors**: Broader geopolitical tensions and uncertainties regarding fiscal policies are contributing to inflationary pressures, leading investors to seek stability in hard assets like gold.

Overall, the narrative underscores that inflation remains a critical concern for the financial markets, influencing interest rate policies and consumer behavior amid a challenging economic landscape.

## Retrieved Documents with Metadata:

**Document 1 (Similarity: 0.5771):**

...

- **Industry:** Oil & Gas Integrated

- **Date:** 2025-05-29

- **Provider:** MT Newswires

- **Content:** Sector Update: Energy Stocks Rise Thursday Afternoon : Energy stocks rose Thursday afternoon with the NYSE Energy Sector Index up 0.1% and the Energy Selec...

---


## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



YOUR WRITTEN RESPONSE HERE

| Criteria           | WITHOUT METADATA                                                                 | WITH METADATA                                                                      |
|--------------------|---------------------------------------------------------------------------------|-----------------------------------------------------------------------------------|
| Clarity            | Usually clear, summarizing main information from returned text.                      | Slightly improved clarity, especially for company/industry-type questions, with context. |
| Detail & Depth     | Constrained only by raw text content of the returned snippets.                        | Can give slightly more specific information by utilizing company/industry from metadata. |
| Use of Context     | Depending solely on the news text material.                                         | Utilizes both semistructured metadata and news article for additional context.              |
| Accuracy & Grounding | From the retrieved text, but risk of uncertainty without context from the metadata. | More explicitly grounded by associating facts with specific companies/industries via metadata. |
| Relevance          | Aligned with the query on the grounds of text semantic search.                          | Retrieval is unchanged but generated answer relevance can be more tangible with clear metadata relations. |
| Narrative Flow     | Text snippet summary consistent.                                             | Can provide a little more controlled or context-aware narrative through the incorporation of metadata. |